In [ ]:
import sqlite3

import duckdb
import pandas as pd

pd.set_option('display.max_colwidth', None)

In [ ]:
conn = sqlite3.connect('2022_race_results.db')

df = pd.read_sql('select * from ga_special_20220503', conn)

conn.close()

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
conn = duckdb.connect('race_results.db')
conn.sql("""
create or replace table race_results 
as
select 
    county,
    precinct,
    input_office,
    office::varchar as office,
    district::varchar as district,
    input_candidate,
    candidate::varchar as candidate,
    party::varchar as party,
    total_votes::int as total_votes,
    vote_type,
    votes::int as votes
from df
""")

In [ ]:
conn.sql("describe race_results;").show()

### Make sure data loaded correctly...

In [ ]:
conn.sql("""
select input_candidate, sum(votes) as votes
from race_results
group by input_candidate
order by input_candidate;
""").show()

### Fix parties...

In [ ]:
conn.sql("""
update race_results
    set party = 'Republican'
where input_candidate ilike '% (Rep)';
""")

In [ ]:
conn.sql("""
update race_results
    set party = 'Democrat'
where input_candidate ilike '% (Dem)';
""")

In [ ]:
conn.sql("""
select party, count(*) as cnt
from race_results
group by party
order by party;
""").show()

### Fix office names...

In [ ]:
conn.sql("""
select input_office, office, district, count(*) as cnt
from race_results
group by input_office, office, district
order by input_office;
""").show()

In [ ]:
conn.sql("""
update race_results
    set office = 'State House',
        district = '45'
where input_office = 'Special State House of Representatives - District 45'
    and office is null;
""")

In [ ]:
conn.sql("""
select input_office, office, district, count(*) as cnt
from race_results
group by input_office, office, district
order by input_office;
""").show()

### Fix candidate names...

In [ ]:
conn.sql("""
update race_results
    set candidate = trim(regexp_replace(input_candidate, '\s+', ' ', 'g'));
""")

In [ ]:
conn.sql("""
select input_candidate, candidate, count(*) as cnt
from race_results
group by input_candidate, candidate
order by input_candidate;
""").show()

In [ ]:
conn.sql("""
update race_results
    set candidate = replace(candidate, ' (Dem)', '')
where office is not null
    and candidate like '% (Dem)';
""")

In [ ]:
conn.sql("""
update race_results
    set candidate = replace(candidate, ' (Rep)', '')
where office is not null
    and candidate like '% (Rep)';
""")

In [ ]:
conn.sql("""
select input_candidate, candidate, count(*) as cnt
from race_results
group by input_candidate, candidate
order by input_candidate;
""").show()

In [ ]:
conn.sql("""
update race_results
    set candidate = replace(candidate, ',', '')
where office is not null
    and candidate like '%,%';
""")

### Run some QC queries...

In [ ]:
conn.sql("""
select county, candidate, party, min(total_votes::int) as total_votes, sum(votes::int) as votes
from race_results
where office is not null
group by county, candidate, party
order by county, candidate;
""").show()

In [ ]:
conn.sql("""
select office, district, candidate, party, sum(votes::int) as votes
from race_results
where office is not null
group by office, district, candidate, party
order by office, district, candidate;
""").show()

In [ ]:
conn.sql("""
create table final_race_results
as
with election_day_votes as
(
    select county, precinct, office, district, party, candidate, votes as election_day_votes
    from race_results
    where office is not null
        and vote_type = 'Election Day Votes'
),
advanced_votes as
(
    select county, precinct, office, district, party, candidate, votes as advanced_votes
    from race_results
    where office is not null
        and vote_type = 'Advanced Voting Votes'
),
absentee_by_mail_votes as
(
    select county, precinct, office, district, party, candidate, votes as absentee_by_mail_votes
    from race_results
    where office is not null
        and vote_type = 'Absentee by Mail Votes'
),
provisional_votes as
(
    select county, precinct, office, district, party, candidate, votes as provisional_votes
    from race_results
    where office is not null
        and vote_type = 'Provisional Votes'
)
select a.*, b.advanced_votes, c.absentee_by_mail_votes, d.provisional_votes
from election_day_votes as a
    inner join advanced_votes as b
        on a.county = b.county
            and a.precinct = b.precinct
            and a.office = b.office
            and coalesce(a.district, '') = coalesce(b.district, '')
            and coalesce(a.party, '') = coalesce(b.party, '')
            and a.candidate = b.candidate
    inner join absentee_by_mail_votes as c
        on a.county = c.county
            and a.precinct = c.precinct
            and a.office = c.office
            and coalesce(a.district, '') = coalesce(c.district, '')
            and coalesce(a.party, '') = coalesce(c.party, '')
            and a.candidate = c.candidate
    inner join provisional_votes as d
        on a.county = d.county
            and a.precinct = d.precinct
            and a.office = d.office
            and coalesce(a.district, '') = coalesce(d.district, '')
            and coalesce(a.party, '') = coalesce(d.party, '')
            and a.candidate = d.candidate
where a.office is not null
order by a.candidate, a.county, a.precinct;
""")

In [ ]:
conn.sql("describe final_race_results;").show()

In [ ]:
conn.sql("""
select *
from final_race_results
limit 10;
""").show()

In [ ]:
conn.sql("""
select 
    candidate, 
    sum(election_day_votes) as election_day_votes,
    sum(advanced_votes) as advanced_votes,
    sum(absentee_by_mail_votes) as absentee_by_mail_votes,
    sum(provisional_votes) as provisional_votes
from final_race_results
group by candidate
order by candidate;
""").show()

In [ ]:
df_final_results = conn.sql("select * from final_race_results;").df()

In [ ]:
df_final_results.head(5).T

In [ ]:
df_final_results.to_csv('../20220503__ga__special__runoff.csv', index=False)